In [2]:
import requests
from bs4 import BeautifulSoup
import time
import re

# Headers para simular um navegador e evitar bloqueios
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

base_url = "https://www.letras.mus.br"
artist_url = f"{base_url}/zeca-pagodinho/"

print(f"Coletando músicas do artista: {artist_url}")

# --- Passo 1: Coleta a lista de músicas do artista ---
try:
    # Envia uma requisição HTTP GET para a URL do artista
    response = requests.get(artist_url, headers=headers)
    response.raise_for_status()  # Se a resposta for um erro (4xx, 5xx), lança uma exceção
except requests.exceptions.RequestException as e:
    # Em caso de erro de conexão, imprime a mensagem e encerra o script
    print(f"Erro ao acessar a URL do artista: {e}")
    exit()

# Analisa o conteúdo HTML da página com a biblioteca BeautifulSoup
soup = BeautifulSoup(response.text, "lxml")

# Localiza todos os links das músicas usando o seletor CSS correto
songs = soup.select("a.songList-table-songName")

if not songs:
    print("Nenhum link de música foi encontrado. Verifique o seletor CSS 'a.songList-table-songName'.")
    exit()

musicas = []
for song in songs:
    titulo = song.text.strip()
    link = song['href']
    musicas.append({'titulo': titulo, 'link': base_url + link})

print(f"{len(musicas)} músicas encontradas. Iniciando a coleta das letras...")

# --- Passo 2: Coleta a letra de cada música ---
corpus = []
for i, musica in enumerate(musicas):
    print(f"[{i+1}/{len(musicas)}] Coletando: {musica['titulo']}...")

    try:
        letra_resp = requests.get(musica['link'], headers=headers)
        letra_resp.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL da letra ({musica['link']}): {e}")
        time.sleep(5)  # Pausa maior em caso de erro para evitar bloqueio
        continue

    letra_soup = BeautifulSoup(letra_resp.text, "lxml")
    
    # Localiza o bloco de texto da letra usando o seletor correto
    letra_div = letra_soup.find("div", class_="lyric-original") 

    if letra_div:
        # Extrai o texto de dentro do bloco, mantendo as quebras de linha
        letra = letra_div.get_text(separator='\n', strip=True)
        
        # --- Passo 3: Normalização dos dados ---
        letra = letra.lower()
        # Remove caracteres que não são letras, números ou espaços.
        letra = re.sub(r'[^a-záéíóúãõâêôçàèìòùü0-9\s]', '', letra)
        # Remove múltiplos espaços e quebras de linha em excesso.
        letra = re.sub(r'\s+', ' ', letra)
        
        corpus.append(letra)
    else:
        print(f"Não foi possível encontrar a letra para '{musica['titulo']}'. Seletor incorreto.")

    time.sleep(2)  # Pausa para evitar sobrecarregar o servidor

# --- Salva as letras em um arquivo de texto ---
with open("corpus.txt", "w", encoding="utf-8") as f:
    for texto in corpus:
        f.write(texto + "\n\n")

print("\nColeta de dados finalizada. Verifique o arquivo 'corpus.txt'.")

Coletando músicas do artista: https://www.letras.mus.br/zeca-pagodinho/
441 músicas encontradas. Iniciando a coleta das letras...
[1/441] Coletando: Ogum (part. Jorge Ben Jor)...
[2/441] Coletando: Mais Feliz...
[3/441] Coletando: Vacilão...
[4/441] Coletando: Deixa a Vida Me Levar...
[5/441] Coletando: Seu Balancê...
[6/441] Coletando: Verdade...
[7/441] Coletando: Quando a Gira Girou...
[8/441] Coletando: Não Sou Mais Disso...
[9/441] Coletando: Maneiras...
[10/441] Coletando: Faixa Amarela...
[11/441] Coletando: Coração Em Desalinho...
[12/441] Coletando: Minha Fé...
[13/441] Coletando: Lama Nas Ruas...
[14/441] Coletando: Pagode do Gago...
[15/441] Coletando: Alma Boêmia...
[16/441] Coletando: Vou Botar Teu Nome Na Macumba...
[17/441] Coletando: Só o Ôme...
[18/441] Coletando: Quem É Ela?...
[19/441] Coletando: Pago Pra Ver...
[20/441] Coletando: Patota do Cosme...
[21/441] Coletando: À Distância...
[22/441] Coletando: A Grande Família (part. Demônios da Garoa)...
[23/441] Coletand